In [1]:
import numpy as np
import pandas as pd
import sys 

# Reading from file

In [2]:
dataset = pd.DataFrame()
i = 0
for i in range(2,66):
    if i != 3:
        data = pd.read_excel('myExcel\\split_'+str(i)+'.xlsx')
        data = data[['product_id','user_id','comment','label','file']]
        dataset = dataset.append(data)
    i += 1

# Data set of each excel file

In [3]:
def excel_dataframe(identifier):
    excel_list = []
    i = 0 
    for item in dataset.values:
        
        if item[4] == identifier:
            excel_list.append(item) 
        
        i += 1
    excel_df =  pd.DataFrame(excel_list, columns=['product_id','user_id','comment','label','file'])
    return excel_df

In [ ]:
#excel_df = excel_dataframe(55)

# SEARCHING IN COLUMN FOR PRODUCT ID

In [ ]:
def get_prod_label(dataset,identifier):
    product_list = []
    #print("_____PRODUCT______")
    for item in dataset.values:
        if  identifier == item[0] :
            product_list.append(('p',int(item[3]),int(item[1])))
    return product_list

In [ ]:
#get_prod_label(dataset,56900)

# SEARCHING IN COLUMN FOR USER ID

In [ ]:
def get_user_label(dataset,identifier):
    user_list = []
    #print("_____USER______")
    for item in dataset.values:
        if  identifier == item[1] :
            user_list.append(('u',int(item[3]),int(item[0])))
    return user_list

In [ ]:
#get_user_label(dataset,1423896)

# user and prod status

In [ ]:
def status(which,dataset,identifier):
    
    result = []
    temp_list = []
    mid = 0
    
    which.lower()
    if which == 'user':
        my_list = get_user_label(dataset,identifier)
        i = 0
        for n in my_list:


            for item in get_prod_label(dataset,my_list[i][2]):
                mid += item[1]
            result.append(mid / len(get_prod_label(dataset,my_list[i][2])) * 100)
            mid = 0
            i += 1

        j=0
        for n  in my_list:
            if result[j] < -50:
                if my_list[j][1] == -1:
                    temp_list.append(0)
                elif my_list[j][1] == 0:
                    temp_list.append(1)
                else:
                    temp_list.append(2)

            if  result[j] > -50 and result[j] < 50:
                if my_list[j][1] == -1:
                    temp_list.append(-1)
                elif my_list[j][1] == 0:
                    temp_list.append(0)
                else:
                    temp_list.append(1)
            if result[j] > 50:
                if my_list[j][1] == -1:
                    temp_list.append(-2)
                elif my_list[j][1] == 0:
                    temp_list.append(-1)
                else:
                    temp_list.append(0)
                    
            j+=1
            
    elif which == 'product':
        my_list = get_prod_label(dataset,identifier)
        mid = 0
        i = 0
        for n in my_list:
            for item in get_user_label(dataset,my_list[i][2]):
                mid += item[1]

            result.append(mid / len(get_user_label(dataset,my_list[i][2])) * 100)
            mid = 0
            i += 1
            
        j=0
        for n  in my_list:
            if result[j] <= -50:
                if my_list[j][1] == -1:
                    temp_list.append(0)
                elif my_list[j][1] == 0:
                    temp_list.append(1)
                else:
                    temp_list.append(2)

            if result[j] > -50 and result[j] < 50:
                if my_list[j][1] == -1:
                    temp_list.append(-1)
                elif my_list[j][1] == 0:
                    temp_list.append(0)
                else:
                    temp_list.append(1)
            if result[j] > 50:
                if my_list[j][1] == -1:
                    temp_list.append(-2)
                elif my_list[j][1] == 0:
                    temp_list.append(-1)
                else:
                    temp_list.append(0)
                    
            j+=1
            
    else:
        sys.exit("just Enter 'user' or 'product'")
        
    if len(my_list) == 0:
        sys.exit("no comment for this product or user")
        

    #Final result
    res = sum(temp_list)  / len(temp_list)
    if res <= -0.5:
        if 'u' in my_list[0]:
            print('Pessimistic person')
        else:
            print('Bad product')
    elif res > -0.5 and res < 0.5:
        if 'u' in my_list[0]:
            print('Realistic person')
        else:
            print('Average product')
    else:
        if 'u' in my_list[0]:
            print('Optimistic person')
        else:
            print('Good product')

In [ ]:
status('user',dataset,1423896)

In [4]:
from keras.models import Sequential , load_model

Using TensorFlow backend.


In [5]:
from __future__ import unicode_literals
from hazm import *

In [6]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

In [7]:
from keras import utils

In [8]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from pickle import dump,load
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [9]:
from keras.layers import Dense , Embedding
from keras.layers import LSTM
from keras.layers.wrappers import Bidirectional

# COMMENT TOKENIZER

In [10]:
tokenizer = Tokenizer(num_words=1000,split="")

In [11]:
def comment_tokenizer(dataset):
    import pandas as pd

    
    #dataset.drop_duplicates(['comment'])
    
    stopword=stopwords_list()
    index=[]
    a=['1','2','3','4','5','6','7','8','9','Q','W','E','R','T','Y','U','I','O','P','A','S','D','F','N','G','H','J','K','L','Z','X','C','V','B','N','M','.',':','?','!','(',')',',',']','.',';','*','q','w','e','r','t','y','u','i','o','p','a','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m','؟']
    h=len(a)
    normalizer = Normalizer()
    
    i=0
    X = []
    for item in dataset['comment']:
        if not isinstance(item, int) and not isinstance(item, float):
            for it in range(0,h) :
                 item=item.replace(a[it], "")
            text = item.replace('\u200c', ' ').replace('\n', '').replace('\r', '').replace('ي', 'ی').replace('ك', 'ک')        
            e = normalizer.normalize(text)
            tokens = word_tokenize(e)
            l3 = [x for x in tokens if x not in stopword]
            X.append(l3)
        else:
            index.append(i)
        i += 1
            
    tokenizer.fit_on_texts(X)
    X = tokenizer.texts_to_sequences(X)
    return X ,index

# label reader

In [13]:
def label_reader(dataset,index):
    y = []
    j=0
    for item in dataset['label']:
        if j not in index:
            y.append(item)
        j += 1
        
    import math
    for n , label in enumerate(y):
        if math.isnan(label):
            y[n] = 0
    return y

In [14]:
X,index = comment_tokenizer(dataset)

In [15]:
y = label_reader(dataset,index)

# CREATING MODEL

In [16]:
def get_ready_train(X,y):
    
    x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.20,random_state=42)
    
    x_train = sequence.pad_sequences(x_train,maxlen=100)
    x_test = sequence.pad_sequences(x_test,maxlen=100)
  
    x_test = np.array(x_test)
    x_train = np.array(x_train)

    
    x_test = np.array(x_test)
    x_train = np.array(x_train)

    y_train = utils.to_categorical(y_train,3) 
    y_test = utils.to_categorical(y_test,3)
    
    
    return x_train , x_test , y_train , y_test 

In [17]:
x_train, x_test , y_train, y_test = get_ready_train(X,y)

In [18]:
def create_model():
    
    model = Sequential()
    model.add(Embedding(10000,64))
    model.add(Bidirectional(LSTM(64, return_sequences=True, dropout=0.2)))
    model.add(Bidirectional(LSTM(25, dropout=0.2)))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    print(model.summary())
    return model

In [19]:
 model = create_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          640000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         66048     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 50)                30800     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 153       
Total params: 737,001
Trainable params: 737,001
Non-trainable params: 0
_________________________________________________________________
None


# training model

In [20]:
training = model.fit(x_train,y_train,epochs=3,batch_size=50, validation_split=0.2 )

C:\Users\ParsIran\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 40210 samples, validate on 10053 samples
Epoch 1/3
40210/40210 [==============================] - 133s 3ms/step - loss: 0.6228 - accuracy: 0.7581 - val_loss: 0.5546 - val_accuracy: 0.7868
Epoch 2/3
40210/40210 [==============================] - 136s 3ms/step - loss: 0.5457 - accuracy: 0.7898 - val_loss: 0.5499 - val_accuracy: 0.7894
Epoch 3/3
40210/40210 [==============================] - 132s 3ms/step - loss: 0.5262 - accuracy: 0.7977 - val_loss: 0.5476 - val_accuracy: 0.7908


# finalize model

In [21]:
final = model.fit(x_train,y_train,epochs=3,batch_size=20)

Epoch 1/3
50263/50263 [==============================] - 277s 6ms/step - loss: 0.5342 - accuracy: 0.7943
Epoch 2/3
50263/50263 [==============================] - 277s 6ms/step - loss: 0.5154 - accuracy: 0.8019
Epoch 3/3
50263/50263 [==============================] - 273s 5ms/step - loss: 0.5012 - accuracy: 0.8075


# testing

In [25]:
predictions = model.predict_classes(x_test)

In [26]:
predictions[1100]

1

In [27]:
y_test[1100]

array([0., 1., 0.], dtype=float32)

In [28]:
confusion_matrix(y_test.argmax(axis=1),predictions)

array([[  91,  711,  319],
       [  59, 7476,  513],
       [  49,  820, 2528]], dtype=int64)

In [29]:
test_score , test_acc = model.evaluate(x_test, y_test, batch_size=64)

12566/12566 [==============================] - 7s 575us/step


In [30]:
test_acc

0.803358256816864

# save model

In [31]:
model.save('sentimentAnalysis.h5')

In [32]:
dump(tokenizer,open('my_simpletokenizer','wb'))

# loading model

In [33]:
sent_model = load_model('sentimentAnalysis.h5')

In [34]:
tokenizer = load(open('my_simpletokenizer','rb'))

# test dataset (UNSEEN)

In [35]:
def making_standard(dataset):
    x_test,index = comment_tokenizer(dataset)
    y_test = label_reader(dataset,index)
    x_test = sequence.pad_sequences(x_test,maxlen=50)
    x_test = np.array(x_test)
    y_test = utils.to_categorical(y_test,3)
    return x_test,y_test

In [36]:
def test_dataset(path):
    data = pd.read_excel(path)
    data = data[['comment','label']]
    xt = []
    yt = []
    xt , yt = making_standard(data)
    answer=sent_model.predict_classes(xt)
    return confusion_matrix(yt.argmax(axis=1),answer) , xt , yt

In [37]:
#conf_matrix , xt , yt = test_dataset('')

In [ ]:
#conf_matrix 

In [ ]:
#test_score , test_acc = sent_model.evaluate(xt, yt, verbose=1, batch_size=64)

In [ ]:
#yt

In [ ]:
#test_acc

# test single comment

In [ ]:
def test_single():
    X=[]
    text = input('comment: ')
    tokens = word_tokenize(text)
    X.append(tokens)
    tokenizer.fit_on_texts(X)
    X = tokenizer.texts_to_sequences(X)
    X = sequence.pad_sequences(X, maxlen=150)
    result = sent_model.predict(X)
    print("neutral  positive   negative")
    print(result)


In [ ]:
while True:
    test_single()
    